In [80]:
import pandas as pd
import numpy as np
import nltk
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import csv
from functools import reduce
import re
import string

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk import word_tokenize, pos_tag_sents, pos_tag
import re
from bs4 import BeautifulSoup
import unicodedata
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation
import os, glob
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
stops = set(stopwords.words("english")) 
wordnet_lemmatizer = WordNetLemmatizer()
from wordcloud import WordCloud
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Phoebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Phoebe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Phoebe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Phoebe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [93]:
labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]
col = ["preprocessed_1", "Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]
folder = "../Annotations/CSVs/"
files = ['usanews.csv', 'foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv', 'nogla0.csv','pew0.csv', 'rae0_0.csv', 
         'rae0_1.csv', 'terror0_0.csv', 'terror0_1.csv']
data = ['preprocessed_1']


In [82]:
df = pd.read_csv(folder + files[0])
df[col]
df[labels] = df[labels].astype("float").replace(r'^\s*$', np.nan, regex=True).fillna(0).replace(np.nan,0)
df[data] = df[data].replace(np.nan, '', regex=True)
df["none"] = (df[labels].max(axis=1) == 0).astype(float)
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,message,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic,cleaned_message,POS,tagCounts,preprocessed_1,none
0,0,0,0,0,0,DEMOCRATS ARE TERRORISTS!,0.0,1.0,0.0,0.0,0.0,0.0,DEMOCRATS ARE TERRORISTS!,"[('DEMOCRATS', 'PROPN'), ('be', 'AUX'), ('TERR...","{'PROPN': 2, 'AUX': 1, 'PUNCT': 1}",democrats terrorists,0.0
1,1,1,1,1,1,"Let's be honest, they aren't voting for Biden,...",0.0,0.0,0.0,0.0,0.0,0.0,"Let us be honest, they are not voting for Bide...","[('let', 'VERB'), ('we', 'PRON'), ('be', 'VERB...","{'VERB': 6, 'PRON': 4, 'ADJ': 3, 'PUNCT': 2, '...",let be honest not vote biden kamala and one ke...,1.0
2,2,2,2,2,2,Where would we be if not for RSBN?,0.0,0.0,0.0,0.0,0.0,0.0,Where would we be if not for RSBN?,"[('where', 'ADV'), ('would', 'AUX'), ('we', 'P...","{'ADV': 1, 'AUX': 1, 'PRON': 1, 'VERB': 1, 'SC...",where be not rsbn,1.0
3,3,3,3,3,3,THEY WON'T BE HAPPY WHEN THEY ARE REGULATED 😛😛😥,0.0,1.0,0.0,0.0,0.0,0.0,THEY Will not BE HAPPY WHEN THEY ARE REGULATED...,"[('they', 'PRON'), ('will', 'AUX'), ('not', 'P...","{'PRON': 2, 'AUX': 2, 'PART': 1, 'VERB': 2, 'A...",not be happy when regulate 😛 😛 😥,0.0
4,4,4,4,4,4,TRUMP 2020,0.0,0.0,0.0,0.0,0.0,0.0,TRUMP 2020,"[('TRUMP', 'PROPN'), ('2020', 'NUM')]","{'PROPN': 1, 'NUM': 1}",trump 2020,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16678,16678,16678,16678,16678,39361,OUR VOTES WILL DRAIN THE SWAMP,0.0,0.0,0.0,0.0,0.0,0.0,OUR VOTES WILL DRAIN THE SWAMP,"[('our', 'PRON'), ('vote', 'NOUN'), ('will', '...","{'PRON': 1, 'NOUN': 2, 'AUX': 1, 'VERB': 1, 'D...",vote drain swamp,1.0
16679,16679,16679,16679,16679,39362,"Biden MAY BE elected president Tuesday, BUT if...",0.0,0.0,0.0,0.0,0.0,0.0,"Biden MAY BE elected president Tuesday, BUT if...","[('Biden', 'PROPN'), ('may', 'AUX'), ('be', 'A...","{'PROPN': 9, 'AUX': 4, 'VERB': 5, 'NOUN': 5, '...",biden elect president tuesday but so ™ t be lo...,1.0
16680,16680,16680,16680,16680,39363,Trump 2020,0.0,0.0,0.0,0.0,0.0,0.0,Trump 2020,"[('trump', 'NOUN'), ('2020', 'NUM')]","{'NOUN': 1, 'NUM': 1}",trump 2020,1.0
16681,16681,16681,16681,16681,39364,I DON'T WANT NO WOMAN FOR PRESIDENT!!!!!,0.0,1.0,0.0,0.0,0.0,0.0,I Do not WANT NO WOMAN FOR PRESIDENT!!!!!,"[('I', 'PRON'), ('do', 'AUX'), ('not', 'PART')...","{'PRON': 1, 'AUX': 1, 'PART': 1, 'VERB': 1, 'D...",not want woman president,0.0


In [83]:
def get_tfidf_vect(documents):
    vectorizer = TfidfVectorizer(ngram_range=(2,3), max_df=0.85, analyzer = 'word', min_df=1, max_features=10000, stop_words='english', lowercase=True)
    vector = vectorizer.fit_transform(documents)
    feature_names = np.array(vectorizer.get_feature_names())
    return pd.DataFrame(vector.todense(), columns=feature_names), vector, feature_names

In [84]:
tfidf_df, vector, features = get_tfidf_vect(df['preprocessed_1'])
tfidf_df[labels] = df[labels]
tfidf_df.head()

,000 die,000 like,000 people,000 tonight,000 tonight brett,000 viewer,0ï âƒ,0ï âƒ 2ï,10 big,10 big guy,...,𝗧𝗥𝗨𝗠𝗣 𝗢𝗡 𝗖𝗛𝗥𝗜𝗦𝗧𝗜𝗔𝗡,𝗧𝗥𝗨𝗠𝗣 𝗨𝗦,𝗧𝗥𝗨𝗦𝗧 𝗗𝗘𝗙𝗘𝗔𝗧,𝗧𝗥𝗨𝗦𝗧 𝗗𝗘𝗙𝗘𝗔𝗧 j𝗢𝗘,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
combined_tfidf = pd.DataFrame(columns= (list(features) + labels))
for label in labels:
    combined_scores = tfidf_df[tfidf_df[label] == 1][features].sum()
    to_append = list(combined_scores) + [0,0,0,0,0,0]
    row = pd.Series(to_append, index = (list(features) + labels))
    combined_tfidf = combined_tfidf.append(row, ignore_index=True)
    combined_tfidf.loc[combined_tfidf.index[-1],label] = 1.0
combined_tfidf

,000 die,000 like,000 people,000 tonight,000 tonight brett,000 viewer,0ï âƒ,0ï âƒ 2ï,10 big,10 big guy,...,𝗧𝗥𝗨𝗠𝗣 𝗢𝗡 𝗖𝗛𝗥𝗜𝗦𝗧𝗜𝗔𝗡,𝗧𝗥𝗨𝗠𝗣 𝗨𝗦,𝗧𝗥𝗨𝗦𝗧 𝗗𝗘𝗙𝗘𝗔𝗧,𝗧𝗥𝗨𝗦𝗧 𝗗𝗘𝗙𝗘𝗔𝗧 j𝗢𝗘,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,1.0,0.0,0.0,0.0,0.0,0.0
1,0.594264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.27735,0.27735,0.0,1.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,1.0,0.0,0.0,0.0
3,0.594264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.27735,0.27735,0.0,0.0,0.0,1.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,1.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,1.0


In [116]:
def get_combined_tfidf_values_per_label(tfidf_df):
    combined_tfidf = pd.DataFrame(columns= (list(features) + labels))
    for label in labels:
        combined_scores = tfidf_df[tfidf_df[label] == 1][features].sum()
        to_append = list(combined_scores) + [0,0,0,0,0,0]
        row = pd.Series(to_append, index = (list(features) + labels))
        combined_tfidf = combined_tfidf.append(row, ignore_index=True)
        combined_tfidf.loc[combined_tfidf.index[-1],label] = 1.0
    return combined_tfidf

In [117]:
for file in files:
    df = pd.read_csv(folder + file)
    df[labels] = df[labels].astype("float").replace(r'^\s*$', np.nan, regex=True).fillna(0).replace(np.nan,0)
    df[data] = df[data].replace(np.nan, '', regex=True)
    df["none"] = (df[labels].max(axis=1) == 0).astype(float)
    
    tfidf_df, vector, features = get_tfidf_vect(df['preprocessed_1'])
    tfidf_df[labels] = df[labels]
    combined_tfidf = get_combined_tfidf_values_per_label(tfidf_df)
    
    combined_tfidf.to_csv("tfidf_per_label" + file)

,Unnamed: 0,000 die,000 like,000 people,000 tonight,000 tonight brett,000 viewer,0ï âƒ,0ï âƒ 2ï,10 big,...,𝗧𝗥𝗨𝗠𝗣 𝗢𝗡 𝗖𝗛𝗥𝗜𝗦𝗧𝗜𝗔𝗡,𝗧𝗥𝗨𝗠𝗣 𝗨𝗦,𝗧𝗥𝗨𝗦𝗧 𝗗𝗘𝗙𝗘𝗔𝗧,𝗧𝗥𝗨𝗦𝗧 𝗗𝗘𝗙𝗘𝗔𝗧 j𝗢𝗘,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,1.0,0.0,0.0,0.0,0.0,0.0
1,1,0.594264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.27735,0.27735,0.0,1.0,0.0,0.0,0.0,0.0
2,2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,1.0,0.0,0.0,0.0
3,3,0.594264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.27735,0.27735,0.0,0.0,0.0,1.0,0.0,0.0
4,4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,1.0,0.0
5,5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,1.0
